# Notebook 13A: Time-Dependent Advection-Diffusion with Physical Units

This notebook extends the basic advection-diffusion tutorial (Notebook 7) by solving in **physical units**. This validates that the units system correctly handles time-dependent problems with mixed advection and diffusion.

## Key Concepts

1. **Physical Units** - Working with dimensional quantities (velocities in cm/year, etc.)
2. **Dimensional Analysis** - Ensuring correct scaling of time-dependent terms
3. **Unit-Aware Time Stepping** - CFL conditions in physical units
4. **Validation** - Comparing dimensional and non-dimensional solutions

## The Advection-Diffusion Equation in Physical Units

We solve:
$$\frac{\partial T}{\partial t} + \mathbf{u} \cdot \nabla T = \kappa \nabla^2 T$$

where:
- $T$ is temperature [K]
- $\mathbf{u}$ is velocity [cm/year]
- $\kappa$ is thermal diffusivity [m²/s]
- $t$ is time [year]

In [1]:
# Fix pyvista for interactive notebooks
import nest_asyncio
nest_asyncio.apply()

In [2]:
import numpy as np
import sympy
import underworld3 as uw
import matplotlib.pyplot as plt

## Create Model First

**Important Pattern**: To use physical units, we must:
1. **Create a `model` object first**
2. **Define the unit system** (length, time, mass scales)
3. **Create meshes and variables**

This ensures the Model owns the unit system and all meshes/variables are consistent.

In [3]:
# Step 1: Create Model FIRST
model = uw.Model()

# Step 2: Define the unit system
# Choose natural scales for a mantle convection problem
L_scale = uw.quantity(2900, "km")      # Mantle depth
t_scale = uw.quantity(1, "Myr")        # Geological timescale
M_scale = uw.quantity(1e24, "kg")      # Large mass scale
T_scale = uw.quantity(1000, "K")       # Temperature scale

model.set_reference_quantities(
    length=L_scale,
    time=t_scale,
    mass=M_scale,
    temperature=T_scale, 
    nondimensional_scaling=True, # The default !
    
)

print("Model created with unit system:")
print(f"  Length scale: {L_scale}")
print(f"  Time scale: {t_scale}")
print(f"  Mass scale: {M_scale}")
print(f"  Temperature scale: {T_scale}")

✓ Units system active with automatic non-dimensionalization
Model created with unit system:
  Length scale: 2900.0 kilometer
  Time scale: 1.0 megayear
  Mass scale: 1e+24 kilogram
  Temperature scale: 1000.0 kelvin


## Problem Setup with Physical Units

Now we can define our physical parameters. The model already knows the unit system.

In [4]:
# Physical parameters with units
L_domain = uw.quantity(2900, "km")           # Domain width
H_domain = uw.quantity(1000, "km")           # Domain height
velocity_phys = uw.quantity(5, "cm/year")    # Horizontal velocity
kappa_phys = uw.quantity(1e-6, "m**2/s")     # Thermal diffusivity
T_left = uw.quantity(273, "K")               # Left boundary temperature (cold)
T_right = uw.quantity(1573, "K")             # Right boundary temperature (hot)

# Time parameters
t_start_phys = uw.quantity(1, "Myr")         # Start after some diffusion
advection_distance = uw.quantity(1000, "km") # How far to advect
t_end_phys = t_start_phys + advection_distance / velocity_phys

print("Physical Parameters:")
print(f"  Domain: {L_domain} × {H_domain}")
print(f"  Velocity: {velocity_phys}")
print(f"  Diffusivity: {kappa_phys}")
print(f"  Temperature range: {T_left} to {T_right}")
print(f"  Time range: {t_start_phys.to('Myr')} to {t_end_phys.to('Myr')}")
print(f"  Duration: {(t_end_phys - t_start_phys).to('Myr')}")

Physical Parameters:
  Domain: 2900.0 kilometer × 1000.0 kilometer
  Velocity: 5.0 centimeter / year
  Diffusivity: 1e-06 meter ** 2 / second
  Temperature range: 273.0 kelvin to 1573.0 kelvin
  Time range: 1.0 megayear to 21.0 megayear
  Duration: 20.0 megayear


## Dimensional Analysis

Let's check the Peclet number and characteristic scales:

In [5]:
# Peclet number: Pe = u*L/κ (ratio of advection to diffusion)
Pe = (velocity_phys * L_domain / kappa_phys).to_reduced_units()

# Diffusion timescale: t_diff = L²/κ
t_diff = (L_domain**2 / kappa_phys).to("Myr")

# Advection timescale: t_adv = L/u
t_adv = (L_domain / velocity_phys).to("Myr")

print(f"Peclet number: {Pe.magnitude:.1f}")
print(f"Diffusion timescale: {t_diff}")
print(f"Advection timescale: {t_adv}")
print(f"\nSince Pe >> 1, advection dominates over diffusion")

Peclet number: 4594.8
Diffusion timescale: 266496.81851598347 megayear
Advection timescale: 58.0 megayear

Since Pe >> 1, advection dominates over diffusion


## Create Mesh in Physical Coordinates

Now that the model exists with a unit system, the mesh will correctly use physical units:

In [6]:
# Step 3: Create mesh (AFTER model and unit system are set up)
res = 10
cellSize_phys = L_domain / res

mesh = uw.meshing.UnstructuredSimplexBox(
    minCoords=(0 * uw.units.km, 0 * uw.units.km),
    maxCoords=(L_domain, H_domain),
    cellSize=cellSize_phys,
    regular=False,
    qdegree=3,
)

# Verify mesh units
x, y = mesh.CoordinateSystem.X
xx = uw.expression_types.UnitAwareExpression(x, uw.units.m)
yy = uw.expression_types.UnitAwareExpression(y, uw.units.m)

print(f"Mesh coordinate units: {uw.get_units(x)}")


Mesh coordinate units: kilometer


In [27]:
uw.function.evaluate(x, mesh.X.coords).max()

UWQuantity(2900000.0, 'meter')

In [8]:
uw.function.evaluate(yy, mesh.X.coords).max()

UWQuantity(1000000.0000000001, 'meter')

In [9]:
t_adv, t_diff

(UWQuantity(58.0, 'megayear'), UWQuantity(266496.81851598347, 'megayear'))

## Analytical Solution in Physical Units

The error function solution for an advected step with diffusion:

$$T(x,t) = T_{\text{left}} + (T_{\text{right}} - T_{\text{left}}) \cdot \frac{1}{2}\left[1 + \text{erf}\left(\frac{x - x_0 - ut}{2\sqrt{\kappa t}}\right)\right]$$

In [10]:
# Convert parameters to base units for SymPy (dimensional analysis)
u_val = velocity_phys.to_base_units().magnitude      # m/s
k_val = kappa_phys.to_base_units().magnitude         # m²/s  
L_val = L_domain.to_base_units().magnitude           # m
t_start_val = t_start_phys.to_base_units().magnitude # s
t_end_val = t_end_phys.to_base_units().magnitude     # s
T_left_val = T_left.magnitude                        # K
T_right_val = T_right.magnitude                      # K

# Define symbolic variables
u_sym, t_sym, x_sym, x0_sym, k_sym = sympy.symbols("u, t, x, x0, k")
T_L, T_R = sympy.symbols("T_L, T_R")

# Analytical solution (normalized to [0,1] first)
T_normalized = (1 + sympy.erf((x_sym - x0_sym - u_sym*t_sym) / (2 * sympy.sqrt(k_sym * t_sym)))) / 2
T_analytical_expr = T_L + (T_R - T_L) * T_normalized

# Initial step position (center of domain after some diffusion)
x0_original = 0.0  # Step started at left boundary at t=0

# At t_start, step has advected to:
x0_at_start = x0_original + u_val * t_start_val

print(f"Step position at t_start: {x0_at_start/1000:.0f} km")
print(f"Step position at t_end: {(x0_original + u_val * t_end_val)/1000:.0f} km")

Step position at t_start: 50 km
Step position at t_end: 1050 km


In [11]:
# Create SymPy expressions for initial and final conditions
# Note: x is already in model units (matches physical units through model's unit system)
T_initial_analytical = T_analytical_expr.subs({
    u_sym: u_val,
    t_sym: t_start_val,
    x_sym: x,  # x coordinate from mesh
    x0_sym: x0_original,
    k_sym: k_val,
    T_L: T_left_val,
    T_R: T_right_val
})

T_final_analytical = T_analytical_expr.subs({
    u_sym: u_val,
    t_sym: t_end_val,
    x_sym: x,  # x coordinate from mesh
    x0_sym: x0_original,
    k_sym: k_val,
    T_L: T_left_val,
    T_R: T_right_val
})

In [12]:
t_now = uw.expression(r"t_\textrm{now}",1, "Current time", units="yr") 
x0_at_start = x0_original + u_val * t_start_val
(t_start_phys * velocity_phys).to("km"), (t_end_phys * velocity_phys).to("km")

(UWQuantity(50.0, 'kilometer'), UWQuantity(1050.0, 'kilometer'))

In [28]:
t_now = uw.expression(r"t_\textrm{now}",1, "Current time", units="yr") 
sqrt_2kt = ((2 * kappa_phys * t_now))**0.5
sqrt_2kt_m = sqrt_2kt.to_base_units() # For later cancellation 
theta = (xx-0*velocity_phys*t_now) / sqrt_2kt_m

T_analytical = sympy.erf(theta)

In [29]:
T_analytical

erf(0.125873126230401*N.x/t_\textrm{now}**0.5)

In [36]:
t_now.sym = uw.quantity(1, "s")
uw.function.evaluate(T_analytical , mesh.X.coords).max()

0.14128604177010687

In [16]:
x0 = uw.quantity(1, uw.units.km)
x1 = uw.quantity(10, "km")

In [17]:
kappa_phys.units

<Unit('meter ** 2 / second')>

In [18]:
uw.function.evaluate((2*t_now.to("s")*kappa_phys)**0.5, mesh.X.coords)

UnitAwareArray([[[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],

                [[23039071.85630532]],



In [19]:
t_now.sym

1

In [20]:
L_scale * x

UWQuantity(2900*N.x, 'kilometer')

In [21]:
(velocity_phys * t_now)._units


<Unit('centimeter')>

In [22]:
uw.check_units_consistency(xx, t_now * velocity_phys)

True

In [23]:
sqrt_2_kt = (2 * kappa_phys * t_start_phys).to_base_units()**0.5
xx + velocity_phys * t_now


UnitAwareExpression(N.x + 5.0*t_\textrm{now} [meter])

In [24]:
uw.function.evaluate(sympy.erf((xx-velocity_phys * t_now)/sqrt_2_kt), mesh.X.coords).max()

-1.0

In [25]:
a = x - velocity_phys * t_now

In [26]:
a.units

AttributeError: 'Add' object has no attribute 'units'

In [ ]:
T_final_analytical

In [ ]:
T_initial_analytical

In [ ]:
T_analytic = (1 + sympy.erf((x_sym - x0_sym - u_sym*t_sym) / (2 * sympy.sqrt(k_sym * t_sym)))) / 2
T_analytical_expr = T_L + (T_R - T_L) * T_normalized


In [ ]:


sqrt_2_kt = uw.quantity(sympy.sqrt((2 * kappa_phys * t_start_phys).to_base_units()),"m")
sqrt_2_kt

## Create Variables and Solver with Units

In [ ]:
# Create mesh variables
T = uw.discretisation.MeshVariable("T", mesh, 1, degree=3, units="K")
v = uw.discretisation.MeshVariable("v", mesh, mesh.dim, degree=1, units="cm/yr")

# Create advection-diffusion solver
adv_diff = uw.systems.AdvDiffusion(
    mesh,
    u_Field=T,
    V_fn=v,
    order=2,
)

# Set diffusivity (physical units!)
adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = kappa_phys

print(f"Diffusivity: {adv_diff.constitutive_model.Parameters.diffusivity}")
print(f"Diffusivity units: {uw.get_units(adv_diff.constitutive_model.Parameters.diffusivity)}")

In [ ]:
# Evaluate boundary conditions from analytical solution
bc_coords = np.array([[0.0, 0.0], [L_val, 0.0]])  # Left and right in meters
bc_values = uw.function.evaluate(T_initial_analytical, bc_coords)

T_left_bc = float(bc_values[0])
T_right_bc = float(bc_values[1])

print(f"Boundary conditions:")
print(f"  Left (x=0): {T_left_bc:.1f} K")
print(f"  Right (x={L_domain}): {T_right_bc:.1f} K")

# Apply boundary conditions
adv_diff.add_dirichlet_bc(T_left_bc, "Left")
adv_diff.add_dirichlet_bc(T_right_bc, "Right")

In [ ]:
# Set velocity field (physical units!)
with uw.synchronised_array_update():
    v.array[:, :, 0] = velocity_phys.magnitude  # x-component
    v.array[:, :, 1] = 0.0                       # y-component

print(f"Velocity field set to {velocity_phys} in x-direction")

In [ ]:
# View solver configuration
adv_diff.view()

## Set Initial Conditions

In [ ]:
# Initialize temperature field from analytical solution
T.array[...] = uw.function.evaluate(T_initial_analytical, T.coords)

print(f"Temperature field initialized")
print(f"  Min: {T.min():.1f} K")
print(f"  Max: {T.max():.1f} K")

In [ ]:
t_test = uw.function.evaluate(x , T.coords)

In [ ]:
T.array[0:10]

In [ ]:
# Visualize initial condition
fig, ax = plt.subplots(figsize=(12, 4))

# Sample along horizontal line at mid-height
sample_x_km = np.linspace(0, L_domain.to("km").magnitude, 100)
sample_y_km = np.zeros_like(sample_x_km) + H_domain.to("km").magnitude / 2
sample_points = np.column_stack([sample_x_km * 1000, sample_y_km * 1000])  # Convert to meters

T_profile = uw.function.evaluate(T.sym, sample_points).squeeze()

ax.plot(sample_x_km, T_profile, 'b-', linewidth=2, label=f'Initial condition (t={t_start_phys.to("Myr")})')


ax.axvline(x=x0_at_start/1000, color='purple', linestyle='--', 
           alpha=0.6, linewidth=2, label=f'Step center at {x0_at_start/1000:.0f} km')
ax.set_xlabel('x (km)')
ax.set_ylabel('Temperature (K)')
ax.set_title(f'Initial Temperature Profile at t = {t_start_phys.to("Myr")}')
ax.grid(True, alpha=0.3)
ax.legend()
plt.show()

## Time Stepping with Physical Units

The solver estimates timesteps based on element-crossing times in physical units:

In [ ]:
# Estimate timestep (in physical units!)
dt_estimate = adv_diff.estimate_dt()
dt_estimate_years = uw.quantity(dt_estimate, "s").to("year")

print(f"Estimated dt: {dt_estimate:.2e} seconds")
print(f"             = {dt_estimate_years}")

# Calculate number of steps
duration = (t_end_phys - t_start_phys).to_base_units().magnitude  # seconds
n_steps = max(1, int(duration / (2 * dt_estimate)))
dt = duration / n_steps
dt_years = uw.quantity(dt, "s").to("year")

print(f"\nSimulation duration: {(t_end_phys - t_start_phys).to('Myr')}")
print(f"Using dt = {dt_years}")
print(f"Number of time steps: {n_steps}")

In [ ]:
# Time stepping loop
model_time = t_start_phys.to_base_units().magnitude  # Start in seconds
time_history = [model_time]

print(f"Time stepping from t = {t_start_phys.to('Myr')} to t = {t_end_phys.to('Myr')}")
print("-" * 60)

for step in range(n_steps):
    # Solve for one time step (dt in seconds)
    adv_diff.solve(timestep=dt)
    model_time += dt
    time_history.append(model_time)
    
    if step % max(1, n_steps // 5) == 0 or step == n_steps - 1:
        t_myr = uw.quantity(model_time, "s").to("Myr")
        print(f"Step {step+1:3d}/{n_steps}: t = {t_myr}")

print("-" * 60)
print(f"Time stepping complete")
print(f"Final time: {uw.quantity(model_time, 's').to('Myr')}")

## Compare with Analytical Solution

In [ ]:
# Sample solutions along horizontal line
T_numerical = uw.function.evaluate(T.sym, sample_points).squeeze()
T_analytical = uw.function.evaluate(T_final_analytical, sample_points).squeeze()

# Calculate error metrics
error = T_numerical - T_analytical
L2_error = np.sqrt(np.mean(error**2))
max_error = np.max(np.abs(error))
relative_error = L2_error / (T_right_val - T_left_val)

print(f"Error Metrics:")
print(f"  L2 Error: {L2_error:.4e} K")
print(f"  Max Error: {max_error:.4e} K")
print(f"  Relative Error: {relative_error:.4%}")

In [ ]:
# Plotting
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Final step position
x_final_km = (x0_original + u_val * t_end_val) / 1000

# Top plot: Solutions comparison
ax1 = axes[0]
ax1.plot(sample_x_km, T_analytical, 'r-', linewidth=2, label='Analytical solution')
ax1.plot(sample_x_km, T_numerical, 'b--', linewidth=2, label='Numerical solution')

ax1.axvline(x=x0_at_start/1000, color='green', 
           linestyle=':', alpha=0.5, linewidth=1.5, 
           label=f'Initial step center ({x0_at_start/1000:.0f} km)')
ax1.axvline(x=x_final_km, color='purple', linestyle='--', alpha=0.6, linewidth=2,
           label=f'Final step center ({x_final_km:.0f} km)')

ax1.set_xlabel('x (km)')
ax1.set_ylabel('Temperature (K)')
ax1.set_title(f'Temperature Profile at t = {t_end_phys.to("Myr")}')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='best')

# Bottom plot: Error distribution
ax2 = axes[1]
ax2.plot(sample_x_km, error, 'g-', linewidth=2)
ax2.axhline(y=0, color='k', linestyle='-', alpha=0.3)
ax2.fill_between(sample_x_km, 0, error, alpha=0.3, color='green')
ax2.axvline(x=x_final_km, color='purple', linestyle='--', alpha=0.3, linewidth=1.5)
ax2.set_xlabel('x (km)')
ax2.set_ylabel('Error (K)')
ax2.set_title(f'Error Distribution (L2 = {L2_error:.2e} K, Relative = {relative_error:.2%})')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Dimensional Analysis Summary

In [ ]:
# Calculate key dimensionless numbers and scales
diffusion_length_end = 2 * (kappa_phys * t_end_phys)**0.5
advection_distance_total = velocity_phys * (t_end_phys - t_start_phys)

print("="*60)
print("DIMENSIONAL ANALYSIS SUMMARY")
print("="*60)
print(f"\nDomain scales:")
print(f"  Length: {L_domain}")
print(f"  Velocity: {velocity_phys}")
print(f"  Diffusivity: {kappa_phys}")
print(f"\nDimensionless numbers:")
print(f"  Peclet number: {Pe.magnitude:.1f} (advection/diffusion ratio)")
print(f"\nCharacteristic timescales:")
print(f"  Diffusion: {t_diff} (L²/κ)")
print(f"  Advection: {t_adv} (L/u)")
print(f"  Simulation: {(t_end_phys - t_start_phys).to('Myr')}")
print(f"\nLength scales at end time:")
print(f"  Diffusion length: {diffusion_length_end.to('km')} (2√(κt))")
print(f"  Advection distance: {advection_distance_total.to('km')} (u·Δt)")
print(f"  Ratio: {(advection_distance_total / diffusion_length_end).to_reduced_units().magnitude:.1f}")
print(f"\nNumerical accuracy:")
print(f"  Relative error: {relative_error:.4%}")
print(f"  Max absolute error: {max_error:.2f} K")
print("="*60)

## Key Takeaways

### 1. **CRITICAL PATTERN: Model → Units → Mesh**

```python
# Step 1: Create Model FIRST
model = uw.Model()

# Step 2: Set up unit system
model.set_reference_quantities(
    length=uw.quantity(2900, "km"),
    time=uw.quantity(1, "Myr"),
    ...
)

# Step 3: THEN create meshes and variables
mesh = uw.meshing.UnstructuredSimplexBox(...)
```

**Why this matters**: The Model owns the unit system. Creating the mesh before the Model means units are ignored!

### 2. **Units work correctly in time-dependent problems**
- The solver handles physical units throughout the time-stepping process
- CFL conditions return timesteps in physical units (seconds)
- Dimensional analysis is automatic

### 3. **Physical insight from dimensional parameters**
Working in physical units makes it easier to interpret:
- Peclet number (advection vs diffusion balance)
- Characteristic timescales (Myr for mantle processes)
- Length scales (diffusion length, advection distance)

### 4. **Correct mesh property access**
- Use `mesh.physical_bounds` or `mesh.physical_extent` (not `get_min_coords()`)

## Exercise 13A.1

Try changing the physical parameters:
1. Increase velocity to 10 cm/year - how does this affect the Peclet number?
2. Change diffusivity to 5e-7 m²/s - what happens to the diffusion timescale?
3. Run for 100 Myr - does the solution remain stable?

## Exercise 13A.2

Verify dimensional consistency:
1. Check that `uw.get_units(adv_diff.constitutive_model.Parameters.diffusivity)` shows correct units
2. Verify that timestep units are seconds: `uw.get_units(dt_estimate)`
3. Compare the final temperature profile with the non-dimensional version from Notebook 7